<a href="https://colab.research.google.com/github/HueyVault/study_NLPs/blob/main/codes/DeepLearnings/11_04_generativeLLM_HF_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 데이터 및 fine tunning 된 모델 셋팅
# !ls -al
# !unzip ./chapter_6_withvLLMme_finetuned_model.zip
# !unzip ./chapter_6_withvLLMme_preprocess.zip
# !ls -al ../input/text2sql/data
# !cp -rf ../input/text2sql/data ./
!ls -al ./data

In [ ]:
import os
# 환경 변수에 따라 파일 경로 설정
if os.getenv('KAGGLE_KERNEL_RUN_TYPE') is not None:
    # Kaggle 환경
    print("kaggle")
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    os.environ['HF_TOKEN']= user_secrets.get_secret("HF_TOKEN")
    os.environ['OPENAI_API_KEY'] = user_secrets.get_secret("OPENAI_API_KEY")

    # file path
    train_file_path =  f'/kaggle/input/house-prices-advanced-regression-techniques/train.csv'
    test_file_path = f'/kaggle/input/house-prices-advanced-regression-techniques/test.csv'

elif 'google.colab' in str(get_ipython()):
    print("colab")
    from google.colab import userdata
    os.environ['HF_TOKEN'] = userdata.get("HF_TOKEN")
    os.environ['OPENAI_API_KEY'] = userdata.get("OPENAI_API_KEY")

else:
    # Docker 환경
    print("local")
    train_file_path = "../../datasets/train.csv"
    test_file_path = "../../datasets/test.csv"


## 자연어 평가
- EM (Extract Match) : 텍스트 매칭(정규식, 의미, 통계 매칭)
- EX (Execution Accuracy) : 개발 환경에서 실행
- GPT-4 활용


In [ ]:
!pip install transformers==4.40.1 bitsandbytes==0.43.1 accelerate==0.29.3 datasets==2.19.0 tiktoken==0.6.0 -qqq

In [ ]:
import pandas as pd
train_file_path='./data/train.csv'
df_text2sql = pd.read_csv(train_file_path)
df_text2sql.head()

In [ ]:
df_text2sql.loc[1,'text']

## fine tuning 된 모델 이용한 응답 만들기

In [ ]:

import torch
from transformers import BitsAndBytesConfig, pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
def make_inference_pipeline(model_id):
    base_model = 'beomi/Yi-Ko-6B'
    # tokenizer
    tokenizer = AutoTokenizer.from_pretrained(base_model)
    # 버전 호환 문제로 삭제
    # # 양자화(모델 사이즈 축소)
    # quantization_config = BitsAndBytesConfig(
    #     load_in_4bit=True,
    #     bnb_4bit_compute_dtype=torch.float16
    # )
    # model
    model = AutoModelForCausalLM.from_pretrained(model_id,
                                                 # quantization_config=quantization_config,
                                                 load_in_4bit=True,
                                                 bnb_4bit_compute_dtype=torch.float16, # 양자화 정의
                                                 device_map='auto'
                                                )

    
    # pipeline : 예측 초기화 설정
    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

    return pipe

In [ ]:
finetuning_model_id = 'HueyVault/yi-ko-6b-text2sql'
hf_pipe = make_inference_pipeline(finetuning_model_id)

In [ ]:
def make_prompt(ddl, question, query=''):
    prompt = f"""당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question을 해결할 수 있는 SQL 쿼리를 생성하세요.
### DDL:
{ddl}
### Question:
{question}
### SQL:
{query}"""
    return prompt

In [ ]:
# 필요한 컬럼 추출해 프롬프트 명령어 만들기
from tqdm import tqdm

for idx, row in tqdm(df_text2sql.iterrows(),
                     total = len(df_text2sql),
                     desc='Generating prompt'): # DataFrame row return
    prompt_command = make_prompt(row['context'],
                         row['question']
                        #  row['answer']
                        )
    df_text2sql.loc[idx, 'prompt'] = prompt_command
    pass
df_text2sql.head()

### 파인튜닝된 모델에 prompt로 질문과 답 얻기

In [ ]:
example = df_text2sql.loc[1,'prompt']

In [ ]:
results = hf_pipe(example, do_sample=False,
       return_full_text=False, max_length=512, truncation=True)
results

In [ ]:
results[0]['generated_text']

In [ ]:
prompts = df_text2sql['prompt'].tolist()
result_gen_sqls = []

for prompt in tqdm(prompts, desc="Generating SQL"):
    result = hf_pipe(prompt,
                    do_sample=False,
                    return_full_text=False,
                    max_new_tokens=512,
                    truncation=True)
    print(result)
    result_gen_sqls.append(result[0]['generated_text'])
df_text2sql['gen_sql'] = result_gen_sqls

In [ ]:
df_text2sql_50 = df_text2sql[:50]
prompts = df_text2sql_50['prompt'].tolist()
len(prompts)

In [ ]:
prompts = df_text2sql_50['prompt'].tolist()
result_gen_sqls = []

for prompt in tqdm(prompts, desc="Generating SQL"):
    result = hf_pipe(prompt,
                    do_sample=False,
                    return_full_text=False,
                    max_new_tokens=512,
                    truncation=True)
    
    result_gen_sqls.append(result[0]['generated_text'])

In [ ]:
df_text2sql_50['gen_sql'] = result_gen_sqls

In [ ]:
df_text2sql_50.head()

In [ ]:
result_gen_sqls[0][0]['generated_text']

### 평가 위한 프롬프트 작성

In [ ]:
import json
import pandas as pd
from pathlib import Path
# gpt_model = "gpt-4-turbo-preview"
gpt_model = "gpt-4o-mini"
def make_requests_for_gpt_evaluation(df, filename, dir='requests'):
  if not Path(dir).exists():
      Path(dir).mkdir(parents=True)
  prompts = []
  for idx, row in df.iterrows():
      prompts.append("""Based on below DDL and Question, evaluate gen_sql can resolve Question. If gen_sql and gt_sql do equal job, return "yes" else return "no". Output JSON Format: {"resolve_yn": ""}""" + f"""
          DDL: {row['context']}
          Question: {row['question']}
          gt_sql: {row['answer']}
          gen_sql: {row['gen_sql']}"""
          )
  jobs = [{"model": gpt_model, "response_format" : { "type": "json_object" }, "messages": [{"role": "system", "content": prompt}]} for prompt in prompts]
  with open(Path(dir, filename), "w") as f:
      for job in jobs:
          json_string = json.dumps(job)
          f.write(json_string + "\n")



In [ ]:
eval_filename = 'text2sql_evaluation.jsonl'

In [ ]:
# 프롬프트 일괄 작성 위한 function 호출과 파일 작성(parallel하게 openai api 호출 위해 )
make_requests_for_gpt_evaluation(df_text2sql_50, eval_filename)

In [ ]:
!wget https://raw.githubusercontent.com/openai/openai-cookbook/refs/heads/main/examples/api_request_parallel_processor.py

In [ ]:
!mkdir results

In [ ]:
!ls -al ./requests

In [ ]:
# GPT-4 평가 수행
eval_filepath = 'text2sql_evaluation.jsonl'
# cl100k_base 토크나이즈 버전

!python api_request_parallel_processor.py \
--requests_filepath requests/{eval_filepath}  \
--save_filepath results/{eval_filepath} \
--request_url https://api.openai.com/v1/chat/completions \
--max_requests_per_minute 2500 \
--max_tokens_per_minute 100000 \
--token_encoding_name cl100k_base \
--max_attempts 5 \
--logging_level 20

### openAI 결과 (results/text2sql_evaluation.jsonl)

In [ ]:
def change_jsonl_to_csv(input_file, output_file, prompt_column="prompt", response_column="response"):
    prompts = []
    responses = []
    with open(input_file, 'r') as json_file:
        for data in json_file:
            prompts.append(json.loads(data)[0]['messages'][0]['content'])
            responses.append(json.loads(data)[1]['choices'][0]['message']['content'])
    df = pd.DataFrame({prompt_column: prompts, response_column: responses})
    df.to_csv(output_file, index=False)
    return df

In [ ]:
evaluation_df = change_jsonl_to_csv(input_file=f'results/{eval_filepath}',
                    output_file=f'results/yi_ko_6b_eval.csv',
                    prompt_column='prompt',
                    response_column='resolve_yn')

In [ ]:
evaluation_read_df = pd.read_csv(f'results/yi_ko_6b_eval.csv')
evaluation_read_df.head()

In [ ]:
evaluation_df['resolve_yn'] = evaluation_df['resolve_yn'].apply(lambda x : json.loads(x)['resolve_yn'])

In [ ]:
correct_sql_number = evaluation_df.query("resolve_yn=='yes'").shape[0]
correct_sql_number

In [ ]:
df_text2sql_50[["answer","gen_sql"]]